# Cube slicing
- Demonstrate how to slice n-dimensional data cubes in to 1d-cubes.
- Show how to convert WGS84 latitude, longitude coordinates into other coordinate systems

## Cube slicing with `xarray`

### Read data
Read data from S3 bucket

In [1]:
import s3fs
import xarray as xr
import rioxarray as rxr
from configparser import ConfigParser
import os
def config(filename, section='s3'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    s3 = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            s3[param[0]] = param[1]
    else:
        raise KeyError(
            f'Section {section} not found in {filename}. Did you specifiy the right path?')
    return s3


try:
    s3_config = config('../../database.ini')
   
except KeyError as er:
    print(er)
    print('Config file not found or malformed, trying with environment variables...')
    if (os.environ.get('S3_FAIRICUBE_STORAGE_BUCKET') is None):
        print('environment variables not set, contact the admin')
    else:
        s3_config = {
            's3_fairicube_storage_bucket': os.environ.get('S3_FAIRICUBE_STORAGE_BUCKET'),
            's3_fairicube_storage_key': os.environ.get('S3_FAIRICUBE_STORAGE_KEY'),
            's3_fairicube_storage_secret': os.environ.get('S3_FAIRICUBE_STORAGE_SECRET')}
        
s3fs_FS = s3fs.S3FileSystem(
    key=s3_config['s3_fairicube_storage_key'],
    secret=s3_config['s3_fairicube_storage_secret'],
)

'Section s3 not found in ../../database.ini. Did you specifiy the right path?'
Config file not found or malformed, trying with environment variables...


In [2]:
# land use
import pandas as pd
da = rxr.open_rasterio(s3fs_FS.open('s3://fairicube/vienna_data/100m/r01_landuse/r01_real_land_use2020_100m_b32_1_1.tif'))
# convert bands into data variables
ds_landuse = xr.merge([da[band].to_dataset(name=f'land_use{band+1}') for band in range(0,32)], compat='override')
ds_landuse

<xarray.Dataset>
Dimensions:      (x: 300, y: 230)
Coordinates:
    band         int64 1
  * x            (x) float64 -1.115e+04 -1.105e+04 ... 1.865e+04 1.875e+04
  * y            (y) float64 3.54e+05 3.538e+05 3.538e+05 ... 3.312e+05 3.31e+05
    spatial_ref  int64 0
Data variables: (12/32)
    land_use1    (y, x) float32 ...
    land_use2    (y, x) float32 ...
    land_use3    (y, x) float32 ...
    land_use4    (y, x) float32 ...
    land_use5    (y, x) float32 ...
    land_use6    (y, x) float32 ...
    ...           ...
    land_use27   (y, x) float32 ...
    land_use28   (y, x) float32 ...
    land_use29   (y, x) float32 ...
    land_use30   (y, x) float32 ...
    land_use31   (y, x) float32 ...
    land_use32   (y, x) float32 ...

Check the `coordinates` of the data cube: `xarray` uses the coordinates of the center point of cell. The extent of the dataset is
```
x_min = -11200.0, 
x_max = 18800.0,
y_min = 331000.0,
y_max = 354000.0
```
with a resolution of `100m`. Therefore the coordinate of the lower-left cell is `(x_min+50,y_min+50) = (-11150.0, 331050.0)` and so on

### Slice cube given point coordinates in WGS84
Let's select a point in OpenStreetMap. OSM uses WGS84, so the coordinates of the point are given in latitude and longitude. In the next cell we convert  the coordinates in the Austrian coordinate reference system, EPSG:31256 using the library `pyproj`.

![example1](./../../images/example1.png)

In [24]:
# convert WGS84 lat, long coordinates in EPSG:31256 (Austrian projection)
import pyproj
lat,long = 48.20254, 16.36874

transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:31256", always_xy=True)

# Perform the transformation
x, y = transformer.transform(long, lat)
print(f"Coordinates in WGS84 (EPSG:4326): lat={lat}, long={long}")
print(f"Coordinates in EPSG:31256: x={x}, y={y}")

Coordinates in WGS84 (EPSG:4326): lat=48.20254, long=16.36874
Coordinates in EPSG:31256: x=2720.9872397864074, y=340459.83491985966


Now we use the transformed `x,y` coordinates to query the data cube.`x,y` will not exactly match any of the coordinates of our data cube, but we can tell `xarray` how to handle them. In this case, we want to find the nearest cell, that is, the cell in which `x,y` falls in.

Alternatively, we can compute the coordinates of the cell with some math.

In [46]:
# look for the nearest coordinates in the cube
cube_1d = ds_landuse.sel(x=x, y=y, method='nearest')
print(cube_1d.coords)

# this is the same as
x_cell = (x//100)*100 + 50 # round up to the closest hundred and offset by 50m
y_cell = (y//100)*100 + 50 
print(x_cell,y_cell)


Coordinates:
    band         int32 1
    x            float64 2.75e+03
    y            float64 3.404e+05
    spatial_ref  int32 0
2750.0 340450.0


Let's check in QGIS that we hit the right cell in the cube: yes! Let's also check that the values of the cube at this cell are the expected ones. Looking at the base map, we see that the cell is mostly covered by a road, and partly by buildings.

![example2](./../../images/example2.png)

In [68]:
# load land use lookup table and rename the data variables to meaningful names
# read LookUpTable (lut)
lut_land_use =pd.read_csv(s3fs_FS.open('s3://fairicube/vienna_data/100m/r01_landuse/lut_realnutzung.csv'))

#update Data variables
variable_list = list(cube_1d.keys()) 

for variable_in_list in variable_list:
    cell_value=lut_land_use[lut_land_use['NUTZUNG_CODE'] == int(variable_in_list[8:])]['NUTZUNG_L1'].values[0]
    cube_1d=cube_1d.rename(name_dict={variable_in_list:cell_value})

In [73]:
# what are the land use classes?
lut_land_use

,NUTZUNG_CODE,NUTZUNG_TEXT,NUTZUNG_L0,NUTZUNG_L1,FLY_indiciator
0,1,Baulandnutzung,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),locker bebautes Wohn(misch)gebiet,2
1,2,Baulandnutzung,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Wohn(misch)gebiet mittlerer Dichte,1
2,3,Baulandnutzung,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),dichtes Wohn(misch)gebiet,1
3,4,Baulandnutzung,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),"grossvolumiger, solidaerer Wohn(misch)bau",1
4,5,Baulandnutzung,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Buero- und Verwaltungsviertel,1
5,6,Baulandnutzung,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",solitaere Handelsstrukturen,1
6,7,Baulandnutzung,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...","Geschaefts-, Kern- u. Mischgebiete",1
7,8,Baulandnutzung,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Mischnutzung wenig dicht,1
8,9,Baulandnutzung,Industrie- und Gewerbenutzung,"Industrie, prod. Gewerbe, Grosshandel inkl. Lager",0
9,10,Baulandnutzung,soziale Infrastruktur,"Kultur, Freizeit, Messe",1


In [81]:
# Check values for selected land use classes
print(f"Share of road: {cube_1d['Strassenraum unbegruent'].values}")
print(f"Share of administrative buildings: {cube_1d['Buero- und Verwaltungsviertel'].values}")

Share of road: 75.0
Share of administrative buildings: 13.0


## Cube slicing with SentinelHub
Get 1d results from SentinelHub data collections.

As an example, we will compute the tree cover density of a point in the Stadtpark Wien. We will query the CLMS High resolution layer Tree Cover Density 2018. The coordinate system of the dataset is EPSG:3035 and the resolution is 10 meters.

In [16]:
# Sentinel Hub authentication
import os
import shapely.geometry
import IPython.display
from sentinelhub import (
    CRS, 
    BBox, 
    DataCollection, 
    SHConfig, 
    SentinelHubCatalog, 
    bbox_to_dimensions, 
    SentinelHubRequest, 
    SentinelHubBYOC,
    MimeType)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")

Let's first choose a point in the Stadpark Wien. We start with lat, long coordinates in WGS84 and then transform them in EPSG:3035

In [36]:
#new lat long coordinates
lat, long = 48.20422901367495, 16.379315653440656
IPython.display.GeoJSON(shapely.geometry.Point(long,lat).__geo_interface__)

<IPython.display.GeoJSON object>

In [37]:
import pyproj
transformer3035 = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3035", always_xy=True)

# Perform the transformation
c1,c2 = transformer3035.transform(long, lat)
print(f"Coordinates of the point in EPSG:3035: {c1,c2}")

Coordinates of the point in EPSG:3035: (4794726.1371622505, 2808334.7818405954)


We define a bounding box centered at the point with a size of 10x10 meters, in EPSG:3035. There is no utility function similar to `xarray.sel(x,y,method='nearest')`, but we can compute the bounding box edges manually.

In [60]:
step = 10/2
bbox_coords = c1-step,c2+step,c1+step,c2-step
bbox = BBox(bbox=bbox_coords, crs=CRS('3035')) # define here your bounding box
bbox_size = bbox_to_dimensions(bbox, resolution=10)
print(f"Point in EPSG:3035: {c1,c2}")
print(f"Bounding box in EPSG:3035: {bbox}")
print(f"Size of the bounding box: {bbox_size}")
IPython.display.GeoJSON(shapely.geometry.box(*bbox.transform_bounds(CRS.WGS84)).__geo_interface__)

Point in EPSG:3035: (4794726.1371622505, 2808334.7818405954)
Bounding box in EPSG:3035: 4794721.1371622505,2808329.7818405954,4794731.1371622505,2808339.7818405954
Size of the bounding box: (1, 1)


<IPython.display.GeoJSON object>

Is the point within the bounding box? Let us check with the original WGS84 coordinates

In [61]:
bbox_wgs84 = bbox.transform_bounds(CRS.WGS84)
# get the middle point of the bounding box and check the distance from the original point
(bbox_wgs84.middle[0] - long, bbox_wgs84.middle[1] - lat) # same coordinates up to the 9th decimal place

(6.039613253960852e-11, -3.6151064364275953e-09)

 Next step: get the data collection from SentinelHub. This is a BYOC collection. We can use the `SentinelHubBYOC` to list all BYOC collections and make a `DataCollection` instance.

In [11]:
# make DataCollections to be used in the SentinelHub request
byoc = SentinelHubBYOC(config=config)
collections_iterator = byoc.iter_collections()
my_collections = list(collections_iterator)
DataCollection_list = []
for collection in my_collections:
    DataCollection_list.append(DataCollection.define_byoc(collection["id"], name=collection["name"]))

In [12]:
for collection in DataCollection.get_available_collections():
    print(collection)

DataCollection.SENTINEL2_L1C
DataCollection.SENTINEL2_L2A
DataCollection.SENTINEL1
DataCollection.SENTINEL1_IW
DataCollection.SENTINEL1_IW_ASC
DataCollection.SENTINEL1_IW_DES
DataCollection.SENTINEL1_EW
DataCollection.SENTINEL1_EW_ASC
DataCollection.SENTINEL1_EW_DES
DataCollection.SENTINEL1_EW_SH
DataCollection.SENTINEL1_EW_SH_ASC
DataCollection.SENTINEL1_EW_SH_DES
DataCollection.DEM
DataCollection.DEM_MAPZEN
DataCollection.DEM_COPERNICUS_30
DataCollection.DEM_COPERNICUS_90
DataCollection.MODIS
DataCollection.LANDSAT_MSS_L1
DataCollection.LANDSAT_TM_L1
DataCollection.LANDSAT_TM_L2
DataCollection.LANDSAT_ETM_L1
DataCollection.LANDSAT_ETM_L2
DataCollection.LANDSAT_OT_L1
DataCollection.LANDSAT_OT_L2
DataCollection.SENTINEL5P
DataCollection.SENTINEL3_OLCI
DataCollection.SENTINEL3_SLSTR
DataCollection.TreeCoverDensity2018_10m_raster
DataCollection.UrbanAtlas2012_10m_raster
DataCollection.environmental_zones_1km
DataCollection.urban_audit_2021_city
DataCollection.UrbanAtlas2018_10m_raster
Da

Finally set up the `evalscript` and the `SentinelHubRequest` to get the data.

In [17]:
evalscript_test = """

//VERSION=3
function setup() {
  return {
    input: ["B01"],
    output: { 
        bands: 1,
        sampleType: "UINT16" // raster format will be UINT16
        }
  };
}

function evaluatePixel(sample) {
  return [sample.B01];
}
"""

request = SentinelHubRequest(
        evalscript=evalscript_test,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.TreeCoverDensity2018_10m_raster,
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.TIFF)
        ],
        bbox=bbox,
        size=bbox_size,
        config=config
    )
data = request.get_data()[0]

In [20]:
# check size of returned data
data.shape # it is a 2d array with 1 row and 1 column

(1, 1)

In [22]:
print(f"share of tree cover density at selected point: {data[0][0]}")

share of tree cover density at selected point: 69
